# Job Dataset Exploration

In [2]:
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

# read the data (about applications)
data = pd.read_csv('dataset/apps.tsv', delimiter='\t')
#display(data.head(10)) # check data being read properly

# extract relevant data
df_ui = data[["UserID", "JobID"]]
#display(df_ui.head(10)) # check data being read properly

# construct dataframe in format (user, item, rating) via column addition
df_ui.insert(2, "rating", np.full((df_ui.shape[0], 1), 1, float))
df_ui = df_ui.rename(columns={"UserID": "user", "JobID": "item"})
#display(df_ui.head(10)) # check data being read properly


# Compute nearest neighbours
userID = 203
itemID = 602298
ratings = df_ui[df_ui['user'] == userID]
ratings2 = ratings[['item', 'rating']]
ratings2 = ratings2.set_index('item')


# We use the collaborative user algorithm UserUser, that use the nearest neighbors 
num_recs = 10  # Number of recommendations to generate
user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(ratings)

selected_movies_useruser = recsys.recommend(203, 10) # generate 10 recommendations for the selected user 

could not load LIBBLAS: Could not find module 'libblas' (or one of its dependencies). Try using the full path with constructor syntax.
